In [2]:
from db import * 
from pageNew import * 
from relation import * 

# Introduccion Actividad 2

En este control trabajaremos con una implementación de componentes básicos de una base de datos.

Se les entregará una implementación sencilla de un storage manager que maneja páginas de disco. Adicionalmente, tendremos una librería que maneja relaciones, y un motor sencillo de bases de datos que permite crear tablas, insertar tuplas, y hacer consultas sencillas.

**IMPORTANTE** : para ejecutar este jupyter notebook deben haber descargado los archivos disponibles en el github y tenerlos en la misma carpeta en la que se encuentra este notebook.   

El flujo de información es el siguiente: los datos se guardan en páginas de disco, y la interfaz de ralciones las carga a través de un iterador a memoria. Esto es un poco distinto a lo que vimos en la clase ya que no habrá un buffer asignado a la base de datos, sino cada instancia de la relación tendrá precisamente una página del disco cargada en la memoria (pueden pensar que para una relación tendremos un frame).

En nuestro caso representaremos las páginas de disco por archivos que contienen 6 líneas donde: 
* las líneas 1 a 5 contienen las casillas
* la línea 6 contiene el string `"<NEXT> next_page"` que indica la siguiente página en el disco; "next_page" es o el nombre de un archivo en el disco, o `"<NULL>"` cuando estamos en la última página 
* cada casilla puede tener una tupla o estar vacía (esto último se represanta con el string `"<EMPTY>"`)
* las tuplas se representarán por strings
* los tipos de datos que puede tener una tupla son int o string que **no contenga espacios** (el espacio lo utilizaremos para separar los distintos datos de una tupla)

**Ejemplo**

Consideremos la relación ``R(a string, b int)`` poblada con los siguientes datos:

a | b
-- | --
hello | 1
bye | 2
hola | 5

Entonces la página en disco que contiene esta relación tendrá el siguiente formato.  


```
hello 1
bye 2
hola 5
<EMPTY>
<EMPTY>
<NEXT> <NULL>
```

En el caso anterior, toda la relación R cabe en una sola página de disco. Veamos ahora el siguiente caso: 

a | b
-- | --
hello | 1
bye | 2
hola | 5
soy | 9
una | 10
relacion | 7
grande | 2

Como el tamaño de pagina soporta un maximo de 5 tuplas, entonces nuestra relacion quedara dividida en dos paginas de disco: `R-1` y `R-2`. 


Primero veamos `R-1`
```
hello 1
bye 2
hola 5
soy 9
una 10
<NEXT> R-2
```
y ahora `R-2`
```
relacion 7
grande 2 
<EMPTY>
<EMPTY>
<EMPTY>
<NEXT> <NULL>
```










## Clase Page

En primer lugar conoceremos la clase `Page` implementada en el archivo `pageNew.py`. Esta clase es la que se encarga de manejar las páginas en las cuales estarán guardados los datos (tuplas).

A continuación se lista la signatura de la clase `Page` que incluye los métodos necesarios para leer las páginas a partir de los archivos y escribir en ellas.
 
```
Page
	Atributos:
	- size (número de tuplas que caben en la página)
	- pname (nómbre del archivo en disco que contiene la página)
	- data = [] (las tuplas de la página, representadas como un string)
	- next = ('<NULL>' o nombre de la siguiente página)

	Métodos:
	- write_page() -- escribir la página al disco
	- read_from_disk() -- leer la página pname desde el disco (pname es un dato del objeto Page)
	- set_next_page(next_pname) -- setea la siguiente página
	- get_next_page_name() -- devuelve la siguiente página, o None si esta no existe
	- insert_into_pos(pos, tuple) -- si la posición pos está vacía, se inserta la tupla tuple en este posición
	- get_element_from_page(pos) -- devuelve la tupla en la posición pos
```


Veamos el uso básico de esta clase:


```py
page = Page('P-1') # Inicializa una página vacía; este objeto todavía no está en el disco
page.read_from_disk() # Carga el contenida de la página P-1 desde el disco, si P-1 existe en el disco
page.write_page() # Escribe la página al disco
page.set_next_page(next_page) # Setea el valor del next = next_page
page.get_next_page_name() # Devuelve la página siguiente, o None si esta no existe
page.insert_into_pos(pos,tuple) # Inserta la tupla tuple (somo un string) a la posición pos, si esta está vacía
page.get_element_from_page(pos) # Devueleve la tupla en la posición pos si se trata de una posición válida (0-4)
```





Por ejemplo, para leer la pagina R-1 (que ya tenemos creada) hacemos: 

In [3]:
# Cargamos una página que ya existe

page = Page('R-1') #Crear la instancia de página
page.read_from_disk() #Cargar la página en disco, si no existe va a tirar un error

# Revisamos que contiene la pagina R-1
print(page.data)

['30 30', '217 3', '2 7', '1 3', '7 2']


## Relaciones

La segunda librería que necesitamos es un manager de tablas. El manager de tablas que usaremos se encuentra en el archivo `relation.py`, y representa la interfaz de una tabla en nuestra base de datos. 

Aquí encontraremos dos clases: `Relation`, que guarda información básica de la relación, y `relIter`, que nos crea un iterador de una relación, y provee los métodos `open()`, `close()` y `next()`, que permiten iterar sobre las tuplas de nuestra relación de la misma manera cómo explicamos en las clases.

```
Relation:
	Atributos:
	- rName -- nombre de la relación; un string 
	- attributes -- lista de nombres de atributos e.g. ['a','b'] 
	- types -- lista de tipos de atributos e.g. ['int','str'] -- solo soportamos int y str 
	- numAttrib = len(attributes) -- número de atributos
	- root_page = rName + '-1' -- nombre de la primera página de la relación (convención R-1, R-2, ... para la relacion R); si la relación está guardada en el disco, se carga la primera página, y si todavía no hay nada en el disco, se crea la primera página
	
	Métodos:
	- get_attribute_types() -- devuelve dicionario con el esquema de la relación (e.g. para R(a int, b str) devuelve {'a': 'int', 'b': 'str'} )
	- get_individual_values(tup) -- si tup es string con una tupla de la relación devuelve el arreglo de valores; e.g. si tup = '1 Hola', R(a int, b str), entonces R.get_individual_values(tup) = [1,'Hola']
	- insert_tuple(tup) -- inserta la tupla tup en la relación (esto deben implementarlo ustedes)
```

```
relIter:
    Atributos:
    - relation -- instancia de la ralación sobre cual vamos a iterar
    - opened_page -- la página de nuestra relación qué tenemos cargada en la RAM
    - current_page -- nombre de la página actual cargada en la memoria (puntero a página)
    - current_pos -- posición en la página actual (puntero a posición)
    
    Métodos:
    - open() -- carga la página raíz de la relación en opened_page; setea los punteros antes de la primera tupla
    - has_next() -- revisa si existe una siguiente tupla; posiciona los punteros current_page y current_pos sobre la siguiente tupla
    - next() -- retorna la siguiente tupla
    - close() -- cierra el iterador
    
```

Veamos el uso básico de la clase `Relation`:


```py
R = Relation('R',['a','b'],['int','int']) # Crea una nueva relación llamada 'R', con esquema R(a int, b int); si la primera página de R (llamada 'R-1' existe en el disco, se asume que la relación ya existe, y se carga desde el disco, si no, se crea la página 'R-1' (sin datos), y escribe al disco)
R.get_attribute_types() # Devuelve el diccionario indicando el tipo de cada atributo... {'a':'int', 'b':'int'} para nuestro ejemplo
R.get_individual_values(tup) # Si 'tup' es una tupla segun el esquema de 'R', se devuelve el diccionario de valores; por ejemplo, R.get_individual_values('1 73') devuelve {'a':1, 'b':73} en nuestro ejemplo
```

Ahora, veamos el uso básico de la clase `relIter` para recorrer las tuplas de la relación R:


```py
iterR = relIter(R)

iterR.open()

t = iterR.next()

while t:
	print(t)
	t = iterR.next()

iterR.close()
```

Veamos un ejemplo: 


In [4]:
R = Relation('R' ,['a','b'], ['int','int']) # Cargamos la relacion R de disco

In [5]:
# Ahora imprimimos por las tuplas de R, esto nos mueve por todas las paginas de R (R-1, R-2, ... , R-n),
# a través del relIter iterR

iterR = relIter(R)

iterR.open()

t = iterR.next()

while t:
    print(t)
    t = iterR.next()

iterR.close()

30 30
217 3
2 7
1 3
7 2
1 1
7 7
1 2
1 8
16 37
15 7
15 4
15 3
15 2
15 1
15 5
16 3
16 2
16 1
15 6
11 22
210 3
218 3
219 3
220 3
123 321
220 7
77 88
88 00
88 99
14 12


## Motor de base de datos

Finalmente, nuestro motor de bases de datos, ubicado en `db.py`, nos permite interactuar con las relaciones, cuyos datos están guardados en la páginas del disco, y los cuales se consiguen ocupando los iteradores.

La base de datos guarda las relaciones en el siguiente objeto:
- `Relations`: un dicionario objetos de la clase `Relation`, indexado por el nombre de la relación (e.g. `{'R': Relation('R',['a','b'],['int','int'])}`)

La base de datos provee las siguientes funcionalidades:

	- create_table(rName, attributes, types) -- se crea un objeto de tipo Relation que ocupa el buffer de la base de datos, y se guarda en el dicionario Relations. Si la relación con el nombre rName ya existe en la base de datos se devuelve un error
	- all_tuples(R) -- devuelve todas las tuplas de la relación R; aquí R es el nombre de la relación, y no una instancia de la clase relation

Veamos un ejemplo de uso: 


In [6]:
# Creando tablas (los archivos de estas dos relaciones están en la carpeta con el código):
create_table('R',['a','b'],['int','int'])
create_table('S',['b','c'],['int','int'])

True

In [8]:
all_tuples('R') # Imprime las tuplas en 'R'

30 30
217 3
2 7
1 3
7 2
1 1
7 7
1 2
1 8
16 37
15 7
15 4
15 3
15 2
15 1
15 5
16 3
16 2
16 1
15 6
11 22
210 3
218 3
219 3
220 3
123 321
220 7
77 88
88 00
88 99
14 12


In [9]:
all_tuples('S') # Imprime todas las tuplas en 'S'

1 1
1 3
4 4
1 8


In [10]:
# Podemos crear una nueva relacion N, con las paginas que creamos anteriormente
# Nuestra relacion sera N(a int, b int, c int)

create_table('N',['a','b','c'],['int','int','int'])

True

In [11]:
# Como nuestras páginas estaban vacías, nuestra relación tambión lo estará
all_tuples('N')

# La actividad

Para esta actividad ustedes deben implementar los siguientes metodos: 




1. **[2 puntos]** Implementar ``insert_tuple(tup)`` en `relation.py` 

    - Para esta primera actividad deben, en primer lugar, revisar si la tupla ya está en la relación. (Hay una clase ya implementada que les puede ayudar en esto.) Si no está, deben insertar la tupla en alguna posición que se encuentre vacía (sea una posición de una página existente marcada con `'<EMPTY>'` o una posición en una página nueva). Si crean una página nueva tienen que linkearla con la anterior. Si exsite una posición vacía en alguna página existente, la tupla se debería insertar allí, sin crear una página nueva. Si su solución siempre crea una página nueva para insertar la tupla, incluso cuando hay posiciones vacías, se aplicará un descuento de 1 punto.
    

2. **[0.5 puntos]** Implementar el método `filtered_tuples(R,cond)` en `db.py`
    - Se trata de la cosulta 'SELECT * FROM R WHERE a = 1'. La condición 'cond' será siempre un string del estilo 'atributo = valor'. Pueden asumir que 'cond' siempre viene bien formateada. El método debería validar si el atributo existe en la relación. 
    
    
3. **[0.5 puntos]** Implementar el método `projected_tuples(R,attribs)` en `db.py`
    - Aquí 'attribs' es una lista de los nombres de atributos de 'R' qué se quieren proyectar. Lo mismo cómo en SQL, no es necesario eliminar los duplicados.
    
    
4. **[1 punto]** Implementar el método `cross(R,S)` en `db.py`
    - Este método debería imprimir las tuplas en el producto cruz de 'R' y 'S'
    
    
5. **[2 puntos]** Implementar el método `nested_loop_join(R,S,join)` en `db.py`
    - Aquí deberían implementar el algoritmo de nested loops join para imprimir todas las tuplas en el join de 'R' y 'S' segun las condiciones especificadas en el 'join'. Aquí 'join' será una lista de pares de atributos que aparecen en 'R' y 'S'. Por ejemplo, nested_loop_join(R,S,join), donde join = [('a','name'),('b','age')], ejecuta la consulta 'SELECT * FROM R,S WHERE R.a = S.name AND R.b = S.age'. La lista 'join' siempre viene bien especificada (i.e. pueden asumir que es una lista de pares de strings). El método debería validar que los atributos existen en sus relaciones respectivas.

**Importante:** Para todos estos métodos, deben ocupar iterador de la relación para acceder a sus tuplas, y no pueden cargar páginas de la relación de otra manera. ¡Si no ocupan los iter se les asignará 0 puntos! Este comentario aplica solo para las actividades 2,3,4,5. Para 1 pueden hacer lo que quieran.





 

## La entrega

Deben entregar un archivo .zip que contenga los archivos .py con las implementaciones pedidas. Ademas debe entregar este jupyter notebook con un demo de código que realice lo siguiente: 

1. Crear una relación nueva  `T(a int, c int, d int)`
2. Usando su método `insert_tuple(tup)`, inserte 7 tuplas en T. Luego de esto, muestre todas las tuplas en T
3. Usando su método `filtered_tuples(R,cond)` realice la consulta `SELECT * FROM T WHERE a=1`
4. Usando su método `projected_tuples(R,attribs)` realice la consulta `SELECT a,d FROM T`
5. Ejecute un producto cruz entre R y T
6. Ejecute un nested loop join entre R y S

Si lo considera necesario, escriba un readme que explique su código y/o como ejecutarlo. 

**Recuerde que los datos de las relaciones R y S ya estan creados y disponibles para su descarga en el github del curso (no deben crearlos ustedes)**



*Escriba aquí su readme!*
> *bla, bla bla*



In [ ]:
## Escriba aqui el demo de uso del metodo insert_tuple(tup)

# Crear la relacion T
# Hacer 7 inserciones
# Mostrar all_tuples('T')


In [ ]:
## Escriba aqui el demo de uso del metodo filtered_tuples(T,cond)

# SELECT * FROM T WHERE a=1

In [ ]:
## Escriba aqui el demo de uso del metodo projected_tuples(T,cond)

# SELECT * FROM T WHERE a=1

In [ ]:
## Escriba aqui el demo de uso del metodo cross(R,S)

# cross('R','S')

In [ ]:
## Escriba aqui el demo de uso del metodo nested_loop_join(R,S)

# nested_loop_join('R','S')
